In [1]:
import numpy as np
import pandas as pd

#### Теперь задача. В этот раз никаких такси и товаров, потренируемся на характеристиках покемонов! В исходном датасете мультииндексов нет, поэтому будем создавать их сами. Данные сохранены в переменную pokemon.

#### Сгруппируйте данные по поколению покемонов (generation), и с помощью value_counts() посчитайте, сколько в каком поколении легендарных покемонов (legendary), а также сколько в этих поколениях нелегендарных покемонов.

**Сначала измените названия исходных столбцов:**

- **пробелы и точки нужно заменить на "_" (напр. Sp. Atk → sp_atk)**
- **приведите все названия к нижнему регистру**
- **колонку "#" переименовать в "id"**

**Полученные результаты запишите в исходный датафрейм pokemon**

In [2]:
pokemon = pd.read_csv('pokemons.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
pokemon.shape

(800, 13)

In [4]:
new_cols_names = {i: '_'.join(i.lower().replace('.', '').split()) if i != '#' else 'id' for i in pokemon.columns}
pokemon.rename(columns=new_cols_names, inplace=True)
pokemon.head()

,id,name,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [5]:
legends = pokemon \
                .groupby(['generation', 'legendary']) \
                .agg({'legendary': 'count'})
legends

legendary
generation legendary           
1          False            160
           True               6
2          False            101
           True               5
3          False            142
           True              18
4          False            108
           True              13
5          False            150
           True              15
6          False             74
           True               8

#### Используйте датафрейм legends, полученный на предыдущем шаге, и измените в нём название столбца, перезаписав его в ту же переменную. Затем используйте unstack, чтобы поместить уровень индекса legendary в уровень оси столбцов. Иными словами, должно получиться две колонки – False & True. Результат сохраните в legends_unstacked.

In [6]:
legends.rename(columns={'legendary': 'legendary_count'}, inplace=True)
legends_unstacked = legends.unstack()
legends_unstacked

legendary_count      
legendary            False True 
generation                      
1                      160     6
2                      101     5
3                      142    18
4                      108    13
5                      150    15
6                       74     8

#### Немного усложним задачу. Теперь попробуем узнать, среди каких типов покемонов и какого поколения больше всего легендарных.

#### Сгруппируйте датасет pokemon по переменным generation и type_1, посчитайте количество легендарных покемонов внутри групп. Приведите данные в формат датафрейма, а затем используйте unstack(). В качестве ответа выберите вид и поколение покемона, среди которых больше всего легендарных.

In [7]:
most_legendary = pokemon \
                        .groupby(['generation', 'type_1']) \
                        .legendary \
                        .value_counts() \
                        .unstack() \
                        .idxmax()
most_legendary

legendary
False     (1, Water)
True     (3, Dragon)
dtype: object

#### Преобразуйте представленные данные в длинный формат и запишите в переменную avocado_agg_long. В качестве индекса используйте type

In [8]:
avocado_agg = pd.DataFrame({'type' : ['conventional', 'organic'],
                            'AvgPrice_2015' : [1.077963, 1.673324],
                            'AvgPrice_2016' : [1.105595, 1.571684],
                            'AvgPrice_2017' : [1.294888, 1.735521],
                            'AvgPrice_2018' : [1.127886, 1.567176],
                            })
avocado_agg

,type,AvgPrice_2015,AvgPrice_2016,AvgPrice_2017,AvgPrice_2018
0,conventional,1.077963,1.105595,1.294888,1.127886
1,organic,1.673324,1.571684,1.735521,1.567176


In [9]:
avocado_agg_long = pd.wide_to_long(avocado_agg, stubnames='AvgPrice', i='type', j='year', sep='_')
avocado_agg_long

,,AvgPrice
type,year,
conventional,2015,1.077963
organic,2015,1.673324
conventional,2016,1.105595
organic,2016,1.571684
conventional,2017,1.294888
organic,2017,1.735521
conventional,2018,1.127886
organic,2018,1.567176


#### Имеется набор данных о супергероях в широком формате. В первой колонке Name находятся их имена, а остальные 167 столбцов — различные характеристики (суперсилы), принимающие значение либо True, либо False. Давайте преобразуем датафрейм так, чтобы "собрать" эти признаки в один столбец под названием superpower, а в качестве значений поместить туда списки имеющихся у того или иного героя суперсил. 

**То есть привести в следующий формат:**

           Name  superpower
568  Spider-Man  [Reflexes, Animal Oriented Powers, Danger Sens..

In [10]:
superheroes = pd.read_csv('superheroes_power_matrix.csv')
superheroes.head()

,Name,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
superheroes.shape

(667, 168)

#### Сначала приведите данные к длинному формату, где единственным идентификатором будет имя героя Name. Новый столбец с названиями суперсил переименуйте в superpower. Полученный датафрейм запишите в superheroes_long.

In [12]:
superheroes_long = superheroes.melt(id_vars='Name', var_name=['superpower'])
superheroes_long.head()

,Name,superpower,value
0,3-D Man,Agility,True
1,A-Bomb,Agility,False
2,Abe Sapien,Agility,True
3,Abin Sur,Agility,False
4,Abomination,Agility,False


In [13]:
superheroes_powers = superheroes_long \
                    .query('value == True') \
                    .groupby('Name') \
                    .superpower \
                    .apply(list) \
                    .to_frame() \
                    .reset_index()
superheroes_powers

,Name,superpower
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,[Lantern Power Ring]
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre..."
...,...,...
662,Yellowjacket II,"[Flight, Energy Blasts, Size Changing]"
663,Ymir,"[Cold Resistance, Durability, Longevity, Super..."
664,Yoda,"[Agility, Stealth, Danger Sense, Marksmanship,..."
665,Zatanna,"[Cryokinesis, Telepathy, Magic, Fire Control, ..."
